In [103]:
from keras.datasets import imdb
from keras import Sequential
from keras.layers import Dense,SimpleRNN,Embedding,Flatten
from keras.utils import pad_sequences
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec,KeyedVectors


In [255]:
# downloading imdb data set and merging train and test to get a net data set

(X_train,y_train),(X_test,y_test) = imdb.load_data()
X= np.concatenate((X_train,X_test),axis=0)
Y= np.concatenate((y_train,y_test),axis=0)


In [256]:
# reverse indexing words and storinf them in decoded review list

word_index = imdb.get_word_index()
reverse_word_index, decoded_review={},[]

for i,j in word_index.items():
    reverse_word_index[j]=i

for z in range(0, len(X)):
    token = ' '.join([reverse_word_index.get(i - 3, '?') for i in X[z]])
    decoded_review.append(token)


In [257]:
# lematizing the words in each sentence of decoded review and storing in final tokens

final_tokens=[]
lemmatizer = WordNetLemmatizer()
for i in range(0,int(len(decoded_review)/100)):
    decoded_review[i]= re.sub('[^a-zA-Z]',' ',decoded_review[i])
    decoded_review[i]= decoded_review[i].lower().split()
    token= [lemmatizer.lemmatize(words) for words in decoded_review[i] if words not in stopwords.words('English')]
    final_tokens.append(token)

Y= Y[0:int(len(decoded_review)/100)]

In [258]:
# training word2vec to get embeddin/gs for each word for every sentence

final_tokens= [i for i in final_tokens if i != []]
model= gensim.models.Word2Vec(final_tokens,min_count=1)

In [259]:


def w2vec(sentence, model):
    w_arr=[]
    for words in sentence:
        if(words in model.wv):
            w_arr.append(model.wv[words])
    # w_arr= model.wv[sentence]

    if(len(w_arr)==0):
        return np.zeros(model.vector_size)
    else:
        return w_arr
    

w2v_arr=[]
for c in final_tokens:
    vectors= w2vec(c, model)
    w2v_arr.append(vectors)

X_padded = pad_sequences(w2v_arr, dtype='float32', padding='post',maxlen=50)

np.shape(X_padded)


(500, 50, 100)

In [260]:
# train test split

X_train,X_test=  X_padded[0:int(0.8*len(X_padded))], X_padded[int(0.8*len(X_padded)):]
Y_train,Y_test= Y[0:int(0.8*len(X_padded))],Y[int(0.8*len(X_padded)):]

shape1, shape2=np.shape(X_train)[1], np.shape(X_train)[2]

In [264]:
# training rnn

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout,LSTM
from tensorflow.keras import Input

model_RNN = Sequential()

# Add a SimpleRNN layer
model_RNN.add(Input(shape=(shape1,shape2)))
model_RNN.add(LSTM(64))

# Optional: Add dropout for regularization
# model_RNN.add(Dropout(0.5))

# Output layer (for binary classification)
model_RNN.add(Dense(1, activation='sigmoid'))


In [265]:
model_RNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_RNN.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.4560 - loss: 0.7007 - val_accuracy: 0.4500 - val_loss: 0.7005
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5254 - loss: 0.6895 - val_accuracy: 0.4750 - val_loss: 0.7002
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5083 - loss: 0.6887 - val_accuracy: 0.5375 - val_loss: 0.6936
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.5498 - loss: 0.6867 - val_accuracy: 0.4750 - val_loss: 0.7035
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5463 - loss: 0.6854 - val_accuracy: 0.4875 - val_loss: 0.6950
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5734 - loss: 0.6873 - val_accuracy: 0.4750 - val_loss: 0.7099
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5635 - loss: 0.6816 - val_accuracy: 0.4875 - val_loss: 0.7000
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5740 - loss: 0.6817 - val_accuracy: 0.4750 - v